# THIS GOES FIRST. BEFORE CLEANING

In [1]:
from processing_eeg_methods.data_utils import (
    get_dataset_basic_info,
)
from processing_eeg_methods.data_loaders import load_data_labels_based_on_dataset
from processing_eeg_methods.share import datasets_basic_infos

subject_id = 1  # Only two things I should be able to change
dataset_name = "braincommand"  # Only two things I should be able to change

dataset_info =get_dataset_basic_info(datasets_basic_infos, dataset_name)

data_path = r"C:\Users\rosit\Documents\workprojects\bci_complete\EEG-Classifiers-Ensemble\Datasets\braincommand_dataset\EEG"

Loop to convert the .csv to .mat file to upload it to EEGLAB

In [2]:
import numpy as np
from scipy.io import savemat

game_mode="singleplayer" # calibration3
subject_id_list = [1]#,8,10,11,12,15,16,17,18,19,20,21,22,23,25,26,27,30,31,33,34,35,36,37,38,39,40,41]
for subject_id in subject_id_list:
    _, X_calibration, y_calibration_original = load_data_labels_based_on_dataset(
        dataset_info,
        subject_id,
        data_path,
        game_mode=game_mode,
    )
    X_calibration = np.transpose(X_calibration, (1, 2, 0))
    data_dict = {"array": X_calibration}
    labels_dict = {"tags": y_calibration_original}
    data_output_path = r"C:\Users\rosit\Documents\MATLAB\2025 clean EEG BrainCommand"
    savemat(fr"{data_output_path}\{subject_id}_{game_mode}_original_delete.mat", data_dict)
    savemat(fr"{data_output_path}\y_{subject_id}_{game_mode}_original_delete.mat", labels_dict)

label 0 is 100
label 1 is 103
label 2 is 51
label 3 is 22
Not setting metadata
276 matching events found
Setting baseline interval to [0.0, 1.296] s
Applying baseline correction (mode: mean)
0 projection items activated


Create the singleplayer_events.

In [1]:
import numpy as np

def create_event_file(game_mode, y_events_original, subject_id):
    # Initialize the first latency
    prev_latency = 0
    
    # Open the file for writing
    file_name = f'C:\\Users\\rosit\\Documents\\MATLAB\\clean EEG BrainCommand\\y_{subject_id}_{game_mode}_events.txt'
    with open(file_name, 'w') as file:
        # Write the header
        file.write('latency\ttype\tduration\n')
        
        for y_event in y_events_original:
            # Calculate the type value
            latency = prev_latency + 1300
            
            # Write the row to the file
            file.write(f'{latency}\t{y_event}\t1300\n')
            
            # Update the latency for the next row
            prev_latency = latency
    
    print(f'File {file_name} created successfully.')

In [4]:
game_mode="singleplayer" # calibration3
subject_id_list = [1,8,10,11,12,15,16,17,18,19,20,21,22,23,25,26,27,30,31,33,34,35,36,37,38,39,40,41]
for subject_id in subject_id_list:
    _, _, y_original = load_data_labels_based_on_dataset(
        dataset_info,
        subject_id,
        data_path,
        game_mode=game_mode,
    )
    
    create_event_file(game_mode, y_original, subject_id)

label 0 is 100
label 1 is 103
label 2 is 51
label 3 is 22
Not setting metadata
276 matching events found
Setting baseline interval to [0.0, 1.296] s
Applying baseline correction (mode: mean)
0 projection items activated
File C:\Users\rosit\Documents\MATLAB\clean EEG BrainCommand\y_1_singleplayer_events.txt created successfully.
label 0 is 85
label 1 is 102
label 2 is 65
label 3 is 23
Not setting metadata
275 matching events found
Setting baseline interval to [0.0, 1.296] s
Applying baseline correction (mode: mean)
0 projection items activated
File C:\Users\rosit\Documents\MATLAB\clean EEG BrainCommand\y_8_singleplayer_events.txt created successfully.
label 0 is 81
label 1 is 102
label 2 is 24
label 3 is 48
Not setting metadata
255 matching events found
Setting baseline interval to [0.0, 1.296] s
Applying baseline correction (mode: mean)
0 projection items activated
File C:\Users\rosit\Documents\MATLAB\clean EEG BrainCommand\y_10_singleplayer_events.txt created successfully.
label 0 is 

# THIS IS AFTER FIXING THE EVENTS

In [4]:
import os
from scipy.io import savemat

def create_event_file(game_mode, y_events_original, subject_id, data_output_path):
    # Initialize the first latency
    prev_latency = 0
    
    # Open the file for writing
    file_name = f'{data_output_path}\\y_{subject_id}_{game_mode}_events.txt'
    with open(file_name, 'w') as file:
        # Write the header
        file.write('latency\ttype\tduration\n')
        
        for y_event in y_events_original:
            # Calculate the type value
            latency = prev_latency + 1300
            
            # Write the row to the file
            file.write(f'{latency}\t{y_event}\t1300\n')
            
            # Update the latency for the next row
            prev_latency = latency
    
    print(f'File {file_name} created successfully.')

def get_player1_turns(file_path):
    player1_turns = []
    
    # Open the file and read the lines
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            if line.startswith("player1_turns"):
                # Extract and parse the player1_turns list portion
                raw_data = line.split("player1_turns,")[1].strip()
                player1_turns = eval(raw_data) # Unsafe but simple for controlled input
    
    return player1_turns

base_path = r"C:\Users\rosit\Documents\workprojects\bci_complete\BrainCommand\assets\game_saved_files\time_and_movement (re-numbered)"
file_pattern = "time_and_movement_singleplayer_sub{:02d}.txt"
game_mode = "singleplayer"
subject_id_list = range(1, 27)
data_output_path = r"C:\Users\rosit\Documents\MCC - Materias\Thesis\Mendeley\Raw\Matlab Extended Version"

# Loop through the file indices
for subject_id in subject_id_list:
    file_name = file_pattern.format(subject_id)
    file_path = os.path.join(base_path, file_name)
    player1_turns = get_player1_turns(file_path)
    
    if player1_turns:
        # Flatten the player1_turns lists
        y_calibration_original = [turn for sublist in player1_turns for turn in sublist]
        
        # Save the flattened list
        labels_dict = {"tags": y_calibration_original}
        output_file = os.path.join(data_output_path, f"y_{subject_id}_{game_mode}.mat")
        savemat(output_file, labels_dict)
        
        
        create_event_file(game_mode, y_calibration_original, subject_id, data_output_path)
        print(f"Processed and saved: {output_file}")
    else:
        print(f"No player1_turns data found in {file_name}")

print("All files processed!")

File C:\Users\rosit\Documents\MCC - Materias\Thesis\Mendeley\Raw\Matlab Extended Version\y_1_singleplayer_events.txt created successfully.
Processed and saved: C:\Users\rosit\Documents\MCC - Materias\Thesis\Mendeley\Raw\Matlab Extended Version\y_1_singleplayer.mat
File C:\Users\rosit\Documents\MCC - Materias\Thesis\Mendeley\Raw\Matlab Extended Version\y_2_singleplayer_events.txt created successfully.
Processed and saved: C:\Users\rosit\Documents\MCC - Materias\Thesis\Mendeley\Raw\Matlab Extended Version\y_2_singleplayer.mat
File C:\Users\rosit\Documents\MCC - Materias\Thesis\Mendeley\Raw\Matlab Extended Version\y_3_singleplayer_events.txt created successfully.
Processed and saved: C:\Users\rosit\Documents\MCC - Materias\Thesis\Mendeley\Raw\Matlab Extended Version\y_3_singleplayer.mat
File C:\Users\rosit\Documents\MCC - Materias\Thesis\Mendeley\Raw\Matlab Extended Version\y_4_singleplayer_events.txt created successfully.
Processed and saved: C:\Users\rosit\Documents\MCC - Materias\Thesi